# Instacart EDA 4  Assignment [ANSWER]
In this assignment you will answer to four individual questions (Questions 0,1,2,3). You can complete any question in any order. **The produced results from one question will not be used on another.**

When you complete this assignment keep this kernel as private, make a succesful commit and add as collaborator the [ISLAB](https://www.kaggle.com/uomislab) .

Note that you will be assessed based on the final version submitted before the deadline.

To answer the questions of this assignment you may create as many code blocks as you wish. In addition, you can use .head( ) or any other method to display your results. **You can use comments to describe the rationale of your solution.** You can also use comments when you use new methods (that we haven't used in our notebooks), new arguments or alternative approaches for the same problem.

**In the case where a code block takes more than 3 minutes to execute, hit the stop button on the left bottom corner and check for any mistakes on your code or your solution in general.**


First load the requested packages and data files for this assignment:

In [ ]:
#load packages
import pandas as pd               # for data manipulation
import matplotlib.pyplot as plt   # for plotting 
import seaborn as sns             # an extension of matplotlib for statistical graphics

#load data
orders = pd.read_csv('../input/orders.csv' )
products = pd.read_csv('../input/products.csv')
order_products_prior = pd.read_csv('../input/order_products__prior.csv')
aisles = pd.read_csv('../input/aisles.csv')

# Question 0
## Q0.0 Create a plot which shows how many orders were placed in each hour of the day. 
- use .groupby( ) method

In [ ]:
orders.groupby('order_hour_of_day')[['order_id']].count().plot.bar()

## Q0.1 What is the percentage allocation of the orders placed from 9 to 5 (09:00-17:00 inclusive) compared to all orders?
- store the final percentage on a variable with name 'pct'
- print the result with the appropriate text

In [ ]:
orders_hour = orders.groupby('order_hour_of_day')[['order_id']].count()
orders_hour.columns = ['volume']
orders_hour = orders_hour.reset_index()
pct = orders_hour[(orders_hour.order_hour_of_day >= 5) & (orders_hour.order_hour_of_day <= 17)].volume.sum() / orders_hour.volume.sum() * 100
print('The orders placed from 9:00 to 17:00 are ' + str(round(pct,2)) + '% of the total orders')

_____________

# Question 1
## Q1.0 Create a DataFrame that keeps all the prior orders and the products that have been purchased

In [ ]:
prd = pd.merge(orders, order_products_prior, on='order_id', how='inner')

## Q1.1 For each product find its average position in the cart (in the orders of all customers).

In [ ]:
item_avg = prd.groupby('product_id')[['add_to_cart_order']].mean()
item_avg.head()

## Q1.2 For a given customer, find the average position of a product on its orders


In [ ]:
position = prd.groupby(['user_id', 'product_id'])[['add_to_cart_order']].mean()
position.columns = ['mean_pos_cart'] 
position.head()

## Q1.3 Select from user with id==35 the product with id==4942. Is the average position of this product for this user greater than the average position from all users?

In [ ]:
position = position.reset_index()
user_pr = position[(position.user_id==35) & (position.product_id==4942)]
user_pr.head()

In [ ]:
user_pr_avg = user_pr.mean_pos_cart.values
user_pr_avg 

In [ ]:
avg_all = item_avg[item_avg.index==4942].add_to_cart_order.values
avg_all

In [ ]:
user_pr_avg > avg_all

_____________

# Question 2
## Q2.0 Create a DataFrame that keeps only the prior orders
- You need to keep only the orders (from orders.csv) and not the products that have been purchased in each order.
- Save the DataFrame as 'orders_prior'

In [ ]:
orders_prior = orders[orders.eval_set== 'prior']
orders_prior.head(20)

## Q2.1 Keep the orders from the customers that have made at least 25 orders
- Use 'orders_prior' DataFrame that you have created on the previous step
- Save the DataFrame as 'orders_25'

In [ ]:
orders_25 = orders_prior.groupby('user_id').filter(lambda x: x.order_number.max() >= 25)
orders_25.head()

In [ ]:
# Sanity check on user_id 28
orders_25[orders_25.user_id==28]

## Q2.2 How many days on average pass for these customers to place an order?

In [ ]:
orders_25.days_since_prior_order.mean()

## Q2.3 Create a histogram for the days that pass since a prior order for the customers with at least 25 orders
- use bins=100

In [ ]:
orders_25.days_since_prior_order.plot.hist(bins=100)

## Q2.4 Calculate the rate of change for the average days since prior order between customers with at least 25 orders & all customers
- Create the following formula:

![](https://latex.codecogs.com/gif.latex?%5Cdpi%7B120%7D%20ROC%3D%5Cfrac%7Bavg%5C_days%5C_since%5C_prior%28%5Cgeqslant%2025%29%20-%20avg%5C_days%5C_since%5C_prior%28ALL%29%7D%7Bavg%5C_days%5C_since%5C_prior%28ALL%29%7D%5Ctimes%20100)

In [ ]:
((orders_25.days_since_prior_order.mean() - orders.days_since_prior_order.mean())/ orders.days_since_prior_order.mean() )* 100

_____________

# Question 3

## Q3.0 Create a DataFrame that keeps all the prior orders and the products that have been purchased

In [ ]:
prd = pd.merge(orders, order_products_prior, on='order_id', how='inner')

## Q3.1 Create a new column which keeps the order_number in reverse order

In [ ]:
prd['order_number_back'] = prd.groupby('user_id')['order_number'].transform(max) - prd.order_number +1 

## Q3.2 Keep only the last 10 orders from each user

In [ ]:
prd10 = prd[prd.order_number_back <= 10]
prd10.head()

## Q3.3 Exclude all the customers who have less than 10 orders in total

In [ ]:
prd10 = prd10.groupby('user_id').filter(lambda x: x.order_id.nunique() == 10)

## Q3.4 For the last 10 orders, find the average basket size of each user.

In [ ]:
basket_size= prd10.groupby(['user_id', 'order_id'])[['product_id']].count()
basket_size.columns = ['volume']
basket_size.head()

In [ ]:
basket_size = basket_size.reset_index()
basket_size_user = basket_size.groupby('user_id')[['volume']].mean()
basket_size_user.head()

## Q3.5 How many customers have average basket size = 1 for their last 10 orders ?

In [ ]:
basket_size_user[basket_size_user.volume==1].count()